In [1]:
import json

with open("data.json", "r") as f:
    data = json.load(f)

# Truy cập các giá trị
timestamps = data["t"]
open_prices = data["o"]
high_prices = data["h"]
low_prices = data["l"]
close_prices = data["c"]
volumes = data["v"]
symbols = data["s"]

In [3]:
import numpy as np
import tensorflow as tf


def prepare_data(data, split_ratio=0.8, window_size=60):
    """
    Prepares data for LSTM model by splitting, normalization, and creating sliding window datasets.

    Args:
        data: Dictionary containing stock data (timestamps, open, high, low, close, volume, symbol).
        split_ratio: Ratio for splitting data into training and testing sets (default: 0.8).
        window_size: Size of the sliding window for creating sequences (default: 60).

    Returns:
        x_train, y_train, x_test, y_test: Tensors containing training and testing data.
    """

    # Extract data from dictionary
    timestamps = data["t"]
    open_prices = data["o"]
    high_prices = data["h"]
    low_prices = data["l"]
    close_prices = data["c"]
    volumes = data["v"]
    symbols = data["s"]

    # Split data into training and testing sets
    split_index = int(len(timestamps) * split_ratio)
    timestamps_train, timestamps_test = (
        timestamps[:split_index],
        timestamps[split_index:],
    )
    open_prices_train, open_prices_test = (
        open_prices[:split_index],
        open_prices[split_index:],
    )
    high_prices_train, high_prices_test = (
        high_prices[:split_index],
        high_prices[split_index:],
    )
    low_prices_train, low_prices_test = (
        low_prices[:split_index],
        low_prices[split_index:],
    )
    close_prices_train, close_prices_test = (
        close_prices[:split_index],
        close_prices[split_index:],
    )
    volumes_train, volumes_test = volumes[:split_index], volumes[split_index:]
    symbols_train, symbols_test = symbols[:split_index], symbols[split_index:]

    # Normalize closing prices (percentage change)
    close_prices_train_norm = (close_prices_train - close_prices_train[0]) / np.float32(
        close_prices_train[0]
    )
    close_prices_test_norm = (close_prices_test - close_prices_test[0]) / np.float32(
        close_prices_test[0]
    )

    # Create sliding window datasets
    x_train, y_train = [], []
    for i in range(len(close_prices_train_norm) - window_size):
        x_train.append(close_prices_train_norm[i : i + window_size])
        y_train.append(close_prices_train_norm[i + window_size])

    x_test, y_test = [], []
    for i in range(len(close_prices_test_norm) - window_size):
        x_test.append(close_prices_test_norm[i : i + window_size])
        y_test.append(close_prices_test_norm[i + window_size])

    # Convert data to tensors
    x_train = tf.convert_to_tensor(x_train)
    y_train = tf.convert_to_tensor(y_train)
    x_test = tf.convert_to_tensor(x_test)
    y_test = tf.convert_to_tensor(y_test)

    return x_train, y_train, x_test, y_test


# Example usage
data = {
    "t": [...],
    "o": [...],
    "h": [...],
    "l": [...],
    "c": [...],
    "v": [...],
    "s": [...],
}  # Your actual data
x_train, y_train, x_test, y_test = prepare_data(data)

# Use x_train, y_train, x_test, y_test for training and testing your LSTM model

TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [2]:
import tensorflow as tf

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.LSTM(128, activation="tanh", return_sequences=True),
        tf.keras.layers.LSTM(64, activation="tanh"),
        tf.keras.layers.Dense(1),
    ]
)

2024-03-21 12:58:25.095649: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-03-21 12:58:25.095675: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-21 12:58:25.095682: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-21 12:58:25.095746: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-21 12:58:25.095762: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
model.compile(optimizer="adam", loss="mse", metrics=["accuracy"])

model.fit(x_train, y_train, epochs=100)

In [ ]:
predictions = model.predict(x_test)